In [1]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gp

In [3]:
ifile_params = ("lowflow_parameters_clean.csv") #catchment characteristics/parameters
vf_params = pd.read_csv(ifile_params, sep=',', index_col=0)
vf_params

,stID,UTM_S_X,UTM_S_Y,UTM_G_X,UTM_G_Y,area,gradient_1085,",gradient_river",height_minimum,height_hypso_10,...,qtt_aug,qtt_sep,qtt_oct,qtt_nov,qtt_dec,qtt_djf,qtt_mam,qtt_jja,qtt_son,qtt_ann
GISID,,,,,,,,,,,,,,,,,,,,,
29,2.11.0,317820.999997,6.919394e+06,320749.800967,6.912807e+06,118.959999,14.33,15.430000,737.0,778.0,...,101.241199,61.948655,40.097098,11.010973,1.608489,2.700414,302.509517,292.518221,113.056726,710.784841
95,2.142.0,338529.999998,6.726448e+06,348095.813860,6.747173e+06,1643.020020,3.80,4.760000,170.0,270.0,...,94.248757,70.586680,65.482156,47.609539,19.529377,38.773152,276.620166,254.304265,183.678375,753.375967
158,2.265.0,294591.999997,6.874376e+06,297604.893072,6.894320e+06,620.090027,4.91,7.750000,338.0,569.0,...,94.446090,55.754340,39.064817,12.267378,2.466287,4.912376,273.678976,262.602652,107.086535,648.280583
168,2.279.0,282323.999996,6.672734e+06,271643.244338,6.688642e+06,435.209991,8.34,9.000000,105.0,180.0,...,100.944595,89.059242,90.441147,70.601822,28.109047,62.374284,343.899147,270.302597,250.102211,926.678223
177,2.291.0,126994.999935,6.895583e+06,129667.974042,6.905215e+06,262.070007,27.24,29.309999,717.0,1226.0,...,261.338736,100.517657,39.517134,6.711838,0.239045,0.350883,157.769787,1308.222887,146.746629,1613.090245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219,307.5.0,452693.999999,7.151617e+06,443505.913795,7.151824e+06,345.619995,5.23,8.190000,310.0,327.0,...,83.201409,85.133415,63.347355,29.495795,18.307632,37.618457,372.953911,341.672979,177.976565,930.221963
2220,307.7.0,448684.999999,7.196271e+06,448270.105104,7.200208e+06,61.369999,19.16,27.120001,480.0,541.0,...,97.227029,123.204449,87.692589,27.714073,8.527037,21.113704,398.232231,620.208865,238.611111,1278.165972
2225,308.1.0,439897.999999,7.124887e+06,433579.504338,7.130803e+06,449.980011,8.85,9.420000,354.0,414.0,...,113.933579,112.036552,77.954321,32.664269,14.023126,28.833964,441.372657,517.392538,222.655142,1210.254253


Creating new dataframe with Østlandet stations and matching catchment parameters from parameterfile

In [18]:
#list with Østlandet stations that matches catchment characterstics from parameter file 


vf_params = vf_params.copy()
vf_params["stID"] = vf_params["stID"].astype(str).str.strip()



ostlandet_list = [
    "12.207.0","19.96.0","73.27.0","19.82.0","19.79.0","19.104.0","16.66.0",
    "16.194.0","16.127.0","16.122.0","15.21.0","12.70.0","12.197.0","2.142.0","12.188.0",
    "12.171.0","12.13.0","11.4.0","3.22.0","2.439.0","2.303.0","2.291.0","2.279.0",
    "2.265.0","2.11.0","311.6.0","12.193.0","313.10.0"
]

#list with wanted stations: 
wanted = []

wanted += [(sid, "Østlandet")  for sid in ostlandet_list]
df_wanted = pd.DataFrame(wanted, columns=["stID", "region"])
df_wanted["stID"] = df_wanted["stID"].astype(str).str.strip()


matched = df_wanted.merge(vf_params, on="stID", how="left")


cols = ["region", "stID"] + [c for c in matched.columns if c not in ("region", "stID")]
matched = matched[cols]


matched.head() #the new dataframe with stations only for Østlandet + catchment characteristics/parameters

,region,stID,UTM_S_X,UTM_S_Y,UTM_G_X,UTM_G_Y,area,gradient_1085,",gradient_river",height_minimum,...,qtt_aug,qtt_sep,qtt_oct,qtt_nov,qtt_dec,qtt_djf,qtt_mam,qtt_jja,qtt_son,qtt_ann
0,Østlandet,12.207.0,181406.999980,6.794136e+06,184603.755945,6.800517e+06,269.880005,14.130000,22.629999,566.0,...,100.228029,66.625229,47.236533,15.234334,2.972272,6.983384,310.174368,286.283983,129.096096,732.537866
1,Østlandet,19.96.0,136290.999968,6.563180e+06,136235.347815,6.563750e+06,0.583800,42.180000,31.639999,580.0,...,133.441681,122.541672,130.500000,94.941666,36.533333,74.233334,457.774998,347.450020,347.983337,1227.441650
2,Østlandet,73.27.0,129377.999946,6.801391e+06,131694.578975,6.796548e+06,30.320000,22.680000,35.110001,1006.0,...,116.669336,85.067334,46.623335,10.474000,0.836000,1.277333,222.489338,712.753984,142.164669,1078.685313
3,Østlandet,19.82.0,127064.999963,6.554680e+06,124041.078747,6.555561e+06,8.907800,25.790001,29.660000,222.0,...,113.414990,106.500000,117.731665,99.673328,67.390002,167.946671,268.354999,285.594989,323.904993,1045.801660
4,Østlandet,19.79.0,103120.999940,6.586966e+06,102149.465597,6.588870e+06,6.300600,108.570000,105.400002,362.0,...,119.747917,112.818756,114.918747,74.118752,35.039581,79.427082,404.462509,324.170837,301.856255,1109.916748


In [19]:
infile_obs = "vf_obs_spec_2.csv" #flow data for whole observation period 
vf_obs_spec_2 = pd.read_csv(infile_obs, sep =',', parse_dates = True, index_col =0)
vf_obs_spec_2 = vf_obs_spec_2.replace(-9999, np.nan)
#vf_obs_spec_2

In [20]:
#flow data created for Østlandet: 


ostlandet_list = [
    "12.207.0","19.96.0","73.27.0","19.82.0","19.79.0","19.104.0","16.66.0",
    "16.194.0","16.127.0","16.122.0","15.21.0","12.70.0","12.197.0","2.142.0","12.188.0",
    "12.171.0","12.13.0","11.4.0","3.22.0","2.439.0","2.303.0","2.291.0","2.279.0",
    "2.265.0","2.11.0","311.6.0","12.193.0","313.10.0"
]

selected_stations = ostlandet_list

#Filtering the dataframe so it only include the Østlandet stations 
vf_selected_ostlandet = vf_obs_spec_2[selected_stations]

#NEw dataframe with only Østlandet stations 
vf_selected_ostlandet.head()

,12.207.0,19.96.0,73.27.0,19.82.0,19.79.0,19.104.0,16.66.0,16.194.0,16.127.0,16.122.0,...,3.22.0,2.439.0,2.303.0,2.291.0,2.279.0,2.265.0,2.11.0,311.6.0,12.193.0,313.10.0
Dato,,,,,,,,,,,,,,,,,,,,,
1900-03-20 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-03-21 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-03-22 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-03-23 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900-03-24 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
startdate = pd.to_datetime("1984-05-15")#.date() #for the new dataframe  Østlandet named vf_selected_ostlandet
enddate = pd.to_datetime("2024-09-15")#.date()
q_prd_ostlandet= vf_selected_ostlandet.loc[startdate:enddate]
q_prd_ostlandet




,12.207.0,19.96.0,73.27.0,19.82.0,19.79.0,19.104.0,16.66.0,16.194.0,16.127.0,16.122.0,...,3.22.0,2.439.0,2.303.0,2.291.0,2.279.0,2.265.0,2.11.0,311.6.0,12.193.0,313.10.0
Dato,,,,,,,,,,,,,,,,,,,,,
1984-05-15 12:00:00,41.438954,14.537512,NaN,8.580346,91.670793,128.504607,112.589920,40.538880,70.429407,107.475285,...,9.355437,53.031326,31.287150,11.545667,16.682602,40.826381,33.090266,24.274383,6.985603,10.172342
1984-05-16 12:00:00,60.460337,42.415209,NaN,33.979884,155.727707,237.287806,140.186713,78.682646,112.918276,213.059038,...,10.164294,107.606045,63.444968,19.555626,27.985725,40.826381,55.633129,25.896341,12.446236,11.771352
1984-05-17 12:00:00,89.288671,47.882835,NaN,33.979884,155.727707,206.673241,149.871904,132.360321,146.902244,203.679318,...,13.613778,122.736272,84.388543,30.443900,36.576828,40.125232,115.735685,30.386596,22.046100,18.618181
1984-05-18 12:00:00,95.966217,66.730042,NaN,62.194932,173.922482,264.174807,249.205489,207.272939,334.090852,497.851211,...,13.164556,96.312665,107.343133,37.876433,44.822151,40.125232,130.139779,30.914606,39.695090,18.618181
1984-05-19 12:00:00,130.091946,66.730042,NaN,56.658885,116.824746,174.019444,121.512058,135.644680,334.090852,194.501614,...,13.613778,83.935667,117.120434,52.016777,48.870032,40.826381,128.027010,31.717755,49.352424,16.454391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-10 12:00:00,NaN,NaN,NaN,45.076788,NaN,NaN,57.159846,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,59.939795,NaN,NaN,NaN
2024-09-11 12:00:00,NaN,NaN,NaN,51.771820,NaN,NaN,47.207480,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,91.789644,NaN,NaN,NaN
2024-09-12 12:00:00,NaN,NaN,NaN,26.810886,NaN,NaN,35.791907,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,82.576514,NaN,NaN,NaN


In [22]:
summer = [5,6,7,8,9] #AM7 for summer season for Østlandet


AM7 = q_prd_ostlandet.loc[q_prd_ostlandet.index.month.isin(summer)].rolling(7).mean().resample("YE").min()#.mean() #gjennomsnittlige minste årlige ukesverdi AM7 tabellen 
AM7

,12.207.0,19.96.0,73.27.0,19.82.0,19.79.0,19.104.0,16.66.0,16.194.0,16.127.0,16.122.0,...,3.22.0,2.439.0,2.303.0,2.291.0,2.279.0,2.265.0,2.11.0,311.6.0,12.193.0,313.10.0
Dato,,,,,,,,,,,,,,,,,,,,,
1984-12-31,2.761249,0.000000,NaN,0.564898,0.440797,0.413506,2.494373,0.550125,4.548049,1.202494,...,1.717835,4.919005,7.992984,18.600589,2.911982,9.678396,8.961335,8.782803,1.212016,1.655157
1985-12-31,2.488827,0.124064,NaN,1.266466,0.993919,3.646687,3.985649,4.591460,3.085696,5.171876,...,3.405659,3.668937,1.949392,3.177044,8.636870,5.952224,2.607229,6.600147,1.672346,8.142687
1986-12-31,1.462201,0.000000,NaN,0.721166,0.251881,0.261410,0.545954,0.287712,2.501627,0.405242,...,0.779635,NaN,10.765409,12.066669,1.556757,9.536346,7.907491,10.749308,0.117551,1.387276
1987-12-31,2.463468,1.358342,NaN,2.087240,0.487051,0.921346,2.552650,1.768405,6.161971,1.818254,...,1.933902,11.540885,16.298923,7.977904,3.740750,14.164559,6.020808,17.066920,0.864061,3.637780
1988-12-31,5.421017,0.000000,NaN,0.984675,0.888509,2.181369,3.382617,2.208521,4.088576,5.060013,...,1.564907,9.592920,4.182653,3.782145,2.906779,NaN,4.770102,16.216317,1.454504,1.588473
1989-12-31,1.846924,0.000000,NaN,0.082175,0.153795,0.321343,1.218931,0.379619,6.040399,1.271015,...,0.494138,5.874582,10.391385,7.478394,1.788413,NaN,7.434769,10.616908,0.088267,0.873757
1990-12-31,2.302312,0.000000,NaN,0.426721,0.314619,0.382535,2.388331,0.925145,3.439176,0.841966,...,0.602795,3.509594,9.937098,17.020191,1.847157,NaN,15.844170,13.137508,0.431850,1.224542
1991-12-31,0.620385,0.000000,7.870081,0.115292,0.195877,0.295816,0.486870,0.166995,2.137531,0.534776,...,0.633532,4.427497,4.616842,4.607570,4.585757,7.456019,7.517661,8.161013,0.196117,1.077616
1992-12-31,1.384594,0.000000,2.900613,0.137055,0.153795,0.290919,0.486870,0.048950,5.262809,0.778785,...,0.628189,2.085134,10.683895,3.826050,1.413348,NaN,5.150431,9.927839,0.002658,0.511910


In [23]:
#log transformation of AM7 
AM7_log = np.log(AM7)          
AM7_log_df = pd.DataFrame

/srv/conda/envs/notebook/lib/python3.10/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


OLS regression with VIF

In [24]:

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Catchment  characteristics/parameters for Østlandet
params = vf_params.copy()
params["stID"] = params["stID"].astype(str).str.strip()

params_ost = params[params["stID"].isin(ostlandet_list)].copy()
params_ost["region"] = "Østlandet"

#merging the  parameters and log AM7
df_ost = params_ost.merge(am7_ost_df, on="stID", how="inner")

#Dataframe for VIF + OLS
exclude = {"stID", "region", "log_AM7"}
num_cols = df_ost.select_dtypes(include=["number"]).columns
predictor_cols_ost = [c for c in num_cols if c not in exclude]

#droping rows with nan
df_ols_ost = df_ost.dropna(subset=predictor_cols_ost + ["log_AM7"]).copy()

#computing VIF  for the predictors with 10 as threshold:
def compute_vif(df, predictors):
    X = sm.add_constant(df[predictors].astype(float))
    vals = []
    for i, var in enumerate(predictors, start=1):
        vals.append((var, variance_inflation_factor(X.values, i)))
    return pd.DataFrame(vals, columns=["variable", "VIF"]).sort_values("VIF", ascending=False)

def vif_filter(df, predictors, thresh=10.0):
    vars_left = predictors.copy()
    while True:
        vif_df = compute_vif(df, vars_left)
        max_vif = vif_df["VIF"].max()
        worst_var = vif_df.loc[vif_df["VIF"].idxmax(), "variable"]
        print(f"Max VIF = {max_vif:.2f} ({worst_var})")
        if max_vif > thresh:
            print(f" -> dropping {worst_var}")
            vars_left.remove(worst_var)
        else:
            print("All VIFs below threshold.")
            break
    final_vif = compute_vif(df, vars_left)
    return vars_left, final_vif

#The VIF filtering
predictors_vif_ost, vif_table_ost = vif_filter(df_ols_ost, predictor_cols_ost, thresh=10.0)

print("\nPredictors after VIF filtering (Østlandet):", predictors_vif_ost)
print("\nFinal VIF table (Østlandet):")
print(vif_table_ost)

#The OLS regression for Østlandet afte the VIF filtering with only VIF less than 10
X_final_ost = sm.add_constant(df_ols_ost[predictors_vif_ost].astype(float))
y_final_ost = df_ols_ost["log_AM7"]

ols_ost = sm.OLS(y_final_ost, X_final_ost).fit()
print("\nOLS summary for Østlandet (after VIF filtering):")
print(ols_ost.summary())


/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Max VIF = inf (UTM_S_X)
 -> dropping UTM_S_X
Max VIF = inf (UTM_S_Y)
 -> dropping UTM_S_Y
Max VIF = inf (UTM_G_X)
 -> dropping UTM_G_X


/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Max VIF = inf (UTM_G_Y)
 -> dropping UTM_G_Y
Max VIF = inf (area)
 -> dropping area
Max VIF = inf (gradient_1085)
 -> dropping gradient_1085


/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


...

Max VIF = inf (p_aug)
 -> dropping p_aug
Max VIF = inf (p_sep)
 -> dropping p_sep
Max VIF = inf (p_oct)
 -> dropping p_oct
Max VIF = inf (p_nov)
 -> dropping p_nov
Max VIF = inf (p_dec)
 -> dropping p_dec
Max VIF = inf (p_djf)
 -> dropping p_djf
Max VIF = inf (p_mam)
 -> dropping p_mam


/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/srv/conda/envs/notebook/lib/python3.10/site-

Max VIF = inf (p_jja)
 -> dropping p_jja
Max VIF = inf (p_son)
 -> dropping p_son
Max VIF = inf (p_ann)
 -> dropping p_ann
Max VIF = inf (q_jan)
 -> dropping q_jan
Max VIF = inf (q_feb)
 -> dropping q_feb
Max VIF = inf (qtt_feb)
 -> dropping qtt_feb
Max VIF = inf (qtt_jan)
 -> dropping qtt_jan
Max VIF = inf (qtt_mar)
 -> dropping qtt_mar
Max VIF = inf (qtt_son)
 -> dropping qtt_son
Max VIF = inf (qtt_jja)
 -> dropping qtt_jja


/srv/conda/envs/notebook/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Max VIF = 692861481133922.50 (q_jja)
 -> dropping q_jja
Max VIF = 191642537334914.72 (qtt_ann)
 -> dropping qtt_ann
Max VIF = 42891425022576.16 (q_son)
 -> dropping q_son
Max VIF = 12025633183899.86 (q_mam)
 -> dropping q_mam
Max VIF = 46198.72 (q_dec)
 -> dropping q_dec
Max VIF = 21674.45 (q_mar)
 -> dropping q_mar
Max VIF = 9200.50 (q_aug)
 -> dropping q_aug
Max VIF = 3441.22 (q_nov)
 -> dropping q_nov
Max VIF = 2130.71 (qtt_mam)
 -> dropping qtt_mam
Max VIF = 1403.43 (q_oct)
 -> dropping q_oct
Max VIF = 1354.37 (q_jul)
 -> dropping q_jul
Max VIF = 426.74 (qtt_dec)
 -> dropping qtt_dec
Max VIF = 382.88 (q_mai)
 -> dropping q_mai
Max VIF = 263.77 (q_sep)
 -> dropping q_sep
Max VIF = 170.70 (qtt_nov)
 -> dropping qtt_nov
Max VIF = 98.53 (q_djf)
 -> dropping q_djf
Max VIF = 74.56 (q_jun)
 -> dropping q_jun
Max VIF = 67.84 (qtt_oct)
 -> dropping qtt_oct
Max VIF = 49.78 (qtt_jul)
 -> dropping qtt_jul
Max VIF = 18.06 (q_ann)
 -> dropping q_ann
Max VIF = 11.28 (qtt_jun)
 -> dropping qtt_jun